# Test library

In [ ]:
import os
import earthaccess
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Login using environment variables
auth = earthaccess.login(strategy="environment")

# Search for Rome area
results = earthaccess.search_data(
    short_name="SRTMGL1",
    version="003",
    bounding_box=(12.35, 41.8, 12.65, 42.0)  # min lon, min lat, max lon, max lat
)

# Download tiles
paths = earthaccess.download(results, "./srtm_tiles")
print(paths)


# Append variables to CVS for training

In [2]:
import os
import re
import zipfile
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import from_origin
from whitebox.whitebox_tools import WhiteboxTools
import earthaccess
from tqdm import tqdm
import contextlib
import io

# ---------------------------
# Tile ID utilities
# ---------------------------
def tile_id_from_coords(lat, lon):
    """Convert coords to tile ID (e.g. N40W106)."""
    import math
    if pd.isna(lat) or pd.isna(lon):
        return None
    ns = "N" if lat >= 0 else "S"
    ew = "E" if lon >= 0 else "W"
    # Use floor for both positive and negative coordinates to handle edge cases properly
    lat_tile = math.floor(lat)
    lon_tile = math.floor(lon)
    return f"{ns}{abs(lat_tile):02d}{ew}{abs(lon_tile):03d}"

# ---------------------------
# DEM Download
# ---------------------------
def download_dem_bbox(min_lon, min_lat, max_lon, max_lat, out_dir="dem_tiles", prefer="SRTMGL1"):
    os.makedirs(out_dir, exist_ok=True)
    earthaccess.login(strategy="environment", persist=True)
    dataset = ("SRTMGL1", "003") if prefer == "SRTMGL1" else ("COPDEM_GLO_30", "001")
    try:
        results = earthaccess.search_data(
            short_name=dataset[0],
            version=dataset[1],
            bounding_box=(min_lon, min_lat, max_lon, max_lat),
            count=10
        )
    except IndexError:
        return []
    if not results or len(results) == 0:
        return []
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf), contextlib.redirect_stderr(buf):
        paths = earthaccess.download(results, out_dir)
    return paths

def download_dem_point(lat, lon, out_dir="dem_tiles", buffer=0.1):
    min_lon = max(-180.0, lon - buffer)
    max_lon = min(180.0, lon + buffer)
    min_lat = max(-90.0, lat - buffer)
    max_lat = min(90.0, lat + buffer)
    paths = download_dem_bbox(min_lon, min_lat, max_lon, max_lat, out_dir=out_dir, prefer="SRTMGL1")
    if paths:
        return paths, "SRTM"
    paths = download_dem_bbox(min_lon, min_lat, max_lon, max_lat, out_dir=out_dir, prefer="COPDEM")
    if paths:
        return paths, "Copernicus"
    return [], "None"

# ---------------------------
# HGT → GeoTIFF
# ---------------------------
def parse_hgt_bounds(hgt_path):
    name = os.path.splitext(os.path.basename(hgt_path))[0]
    m = re.match(r'([NS])(\d{1,2})([EW])(\d{1,3})', name, re.IGNORECASE)
    if not m:
        raise ValueError(f"Cannot parse HGT name: {hgt_path}")
    lat_sign = 1 if m.group(1).upper() == 'N' else -1
    lon_sign = 1 if m.group(3).upper() == 'E' else -1
    import math
    lat0 = lat_sign * math.floor(int(m.group(2)))
    lon0 = lon_sign * math.floor(int(m.group(4)))
    west, south = float(lon0), float(lat0)
    east, north = west + 1.0, south + 1.0
    return west, south, east, north

def hgt_to_gtiff(hgt_path, tif_path):
    west, south, east, north = parse_hgt_bounds(hgt_path)
    nbytes = os.path.getsize(hgt_path)
    side = int(np.sqrt(nbytes // 2))
    if side not in (3601, 1201):
        raise ValueError(f"Unexpected HGT side length: {side}")
    data = np.fromfile(hgt_path, dtype=">i2").reshape((side, side))
    data = data[:-1, :-1]
    res = 1.0 / (side - 1)
    transform = from_origin(west, north, res, res)
    profile = {
        "driver": "GTiff",
        "height": data.shape[0],
        "width": data.shape[1],
        "count": 1,
        "dtype": "int16",
        "crs": "EPSG:4326",
        "transform": transform,
        "nodata": -32768,
        "tiled": True,
        "compress": "LZW"
    }
    with rasterio.open(tif_path, "w", **profile) as dst:
        dst.write(data, 1)

def prepare_tif(path):
    """Unpack zip/HGT and convert to GeoTIFF. Remove raw files after processing."""
    if path.lower().endswith(".tif"):
        return os.path.abspath(path)
    if path.lower().endswith(".zip"):
        tif_out, hgt_out = None, None
        with zipfile.ZipFile(path, "r") as z:
            tifs = [m for m in z.namelist() if m.lower().endswith(".tif")]
            if tifs:
                tif_out = os.path.join(os.path.dirname(path), os.path.basename(tifs[0]))
                if not os.path.exists(tif_out):
                    z.extract(tifs[0], os.path.dirname(path))
                tif_out = os.path.abspath(tif_out)
            else:
                hgts = [m for m in z.namelist() if m.lower().endswith(".hgt")]
                if hgts:
                    hgt_out = os.path.join(os.path.dirname(path), os.path.basename(hgts[0]))
                    if not os.path.exists(hgt_out):
                        z.extract(hgts[0], os.path.dirname(path))
                    tif_out = hgt_out.replace(".hgt", ".tif")
                    if not os.path.exists(tif_out):
                        hgt_to_gtiff(hgt_out, tif_out)
                    try:
                        os.remove(hgt_out)
                    except PermissionError:
                        pass
                    tif_out = os.path.abspath(tif_out)
        try:
            os.remove(path)
        except PermissionError:
            pass
        if tif_out:
            return tif_out
        else:
            raise FileNotFoundError(f"No .tif or .hgt in {path}")
    raise FileNotFoundError(f"Unsupported DEM format: {path}")

# ---------------------------
# Whitebox + helpers
# ---------------------------
wbt = WhiteboxTools()
wbt.verbose = False

def valid_raster(path):
    """Check if a raster exists, non-empty, and can be opened by rasterio."""
    if not path or not os.path.exists(path) or os.path.getsize(path) == 0:
        return False
    try:
        with rasterio.open(path) as src:
            _ = src.count
        return True
    except Exception:
        return False

def run_whitebox(tif_file, need_slope=False, need_aspect=False, need_geomorph=False, slope_dir=None, aspect_dir=None, geomorph_dir=None):
    tif_file = os.path.abspath(tif_file).replace("\\", "/")
    base_name = os.path.splitext(os.path.basename(tif_file))[0]
    
    # Create output paths in subfolders
    slope_tif = os.path.join(slope_dir, f"{base_name}_slope.tif") if slope_dir else f"{os.path.splitext(tif_file)[0]}_slope.tif"
    aspect_tif = os.path.join(aspect_dir, f"{base_name}_aspect.tif") if aspect_dir else f"{os.path.splitext(tif_file)[0]}_aspect.tif"
    geomorph_tif = os.path.join(geomorph_dir, f"{base_name}_geomorph.tif") if geomorph_dir else f"{os.path.splitext(tif_file)[0]}_geomorph.tif"
    
    if need_slope and not valid_raster(slope_tif):
        wbt.slope(dem=tif_file, output=slope_tif, zfactor=1.0, units="degrees")
    if need_aspect and not valid_raster(aspect_tif):
        wbt.aspect(dem=tif_file, output=aspect_tif)
    if need_geomorph and not valid_raster(geomorph_tif):
        wbt.geomorphons(dem=tif_file, output=geomorph_tif, search=50, threshold=0.0, forms=True)
    return (
        tif_file,
        slope_tif if valid_raster(slope_tif) else None,
        aspect_tif if valid_raster(aspect_tif) else None,
        geomorph_tif if valid_raster(geomorph_tif) else None
    )

# ---------------------------
# Extract raster value
# ---------------------------
def extract_value(raster, lat, lon):
    if not valid_raster(raster):
        return None
    try:
        with rasterio.open(raster) as src:
            nd = src.nodata
            for val in src.sample([(lon, lat)]):
                v = float(val[0])
                if np.isnan(v) or (nd is not None and v == nd):
                    return None
                return v
    except Exception:
        return None

# ---------------------------
# Main pipeline
# ---------------------------
def enrich_csv(input_csv, output_csv, out_dir="dem_tiles", download_tiles=True, variables=["dem", "slope", "aspect", "geomorphons"], verbose=True):
    """
    Enhanced version with detailed error logging and failure tracking.
    
    Args:
        input_csv: Input CSV file path
        output_csv: Output CSV file path
        out_dir: Directory for DEM tiles
        download_tiles: Whether to download DEM tiles (if False, only use existing files)
        variables: List of variables to append. Options: ["dem", "slope", "aspect", "geomorphons"]
        verbose: Print detailed progress and error information
    """
    import logging
    from datetime import datetime
    
    # Setup logging
    if verbose:
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logger = logging.getLogger(__name__)
    else:
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.WARNING)
    
    os.makedirs(out_dir, exist_ok=True)
    
    # Create subfolders for different file types
    dem_dir = os.path.join(out_dir, "dem")
    slope_dir = os.path.join(out_dir, "slope")
    aspect_dir = os.path.join(out_dir, "aspect")
    geomorph_dir = os.path.join(out_dir, "geomorphons")
    
    os.makedirs(dem_dir, exist_ok=True)
    os.makedirs(slope_dir, exist_ok=True)
    os.makedirs(aspect_dir, exist_ok=True)
    os.makedirs(geomorph_dir, exist_ok=True)
    
    # Derive individual boolean flags from variables list
    generate_dem = "dem" in variables
    generate_slope = "slope" in variables
    generate_aspect = "aspect" in variables
    generate_geomorphons = "geomorphons" in variables
    
    # Validate variables list
    valid_variables = ["dem", "slope", "aspect", "geomorphons"]
    invalid_vars = [var for var in variables if var not in valid_variables]
    if invalid_vars:
        raise ValueError(f"Invalid variables: {invalid_vars}. Valid options are: {valid_variables}")
    
    # Validate input file
    if not os.path.exists(input_csv):
        raise FileNotFoundError(f"Input CSV file not found: {input_csv}")
    
    logger.info(f"Loading CSV: {input_csv}")
    df = pd.read_csv(input_csv)
    df = df[df["y"].between(-56, 60)]

    logger.info(f"Loaded {len(df)} rows")
    
    # Validate required columns
    required_cols = ["x", "y"]
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Initialize columns based on requested variables
    base_cols = []
    if generate_dem:
        base_cols.extend(["dem", "dem_source"])
    if generate_slope:
        base_cols.append("slope")
    if generate_aspect:
        base_cols.append("aspect")
    if generate_geomorphons:
        base_cols.extend(["geomorphon", "geomorphon_class"])
    
    for col in base_cols:
        if col not in df.columns:
            df[col] = None
    
    # Add failure tracking columns for requested variables
    failure_cols = []
    if generate_dem:
        failure_cols.append("dem_failure")
    if generate_slope:
        failure_cols.append("slope_failure")
    if generate_aspect:
        failure_cols.append("aspect_failure")
    if generate_geomorphons:
        failure_cols.append("geomorphon_failure")
    
    for col in failure_cols:
        if col not in df.columns:
            df[col] = None
    
    # Statistics tracking
    stats = {
        "total_rows": len(df),
        "invalid_coords": 0,
        "missing_tiles": 0,
        "extraction_failures": 0
    }
    
    # Add variable-specific stats based on requested variables
    if generate_dem:
        stats["successful_dem"] = 0
    if generate_slope:
        stats["successful_slope"] = 0
    if generate_aspect:
        stats["successful_aspect"] = 0
    if generate_geomorphons:
        stats["successful_geomorphon"] = 0

    # Step 1: Collect per-tile needs
    logger.info("Step 1: Scanning CSV for tile requirements...")
    tile_needs = {}
    invalid_coord_rows = []
    
    for i, row in tqdm(df.iterrows(), total=len(df), desc="Scanning CSV"):
        lat, lon = row["y"], row["x"]
        
        # Check for invalid coordinates
        if pd.isna(lat) or pd.isna(lon):
            invalid_coord_rows.append(i)
            df.at[i, "dem_failure"] = "Invalid coordinates: NaN values"
            stats["invalid_coords"] += 1
            continue
            
        try:
            tid = tile_id_from_coords(lat, lon)
            if tid is None:
                invalid_coord_rows.append(i)
                df.at[i, "dem_failure"] = f"Invalid coordinates: lat={lat}, lon={lon}"
                stats["invalid_coords"] += 1
                continue
        except Exception as e:
            invalid_coord_rows.append(i)
            df.at[i, "dem_failure"] = f"Coordinate conversion error: {str(e)}"
            stats["invalid_coords"] += 1
            continue
            
        base = os.path.join(dem_dir, tid)
        slope_file = os.path.join(slope_dir, f"{tid}_slope.tif")
        aspect_file = os.path.join(aspect_dir, f"{tid}_aspect.tif")
        geomorph_file = os.path.join(geomorph_dir, f"{tid}_geomorph.tif")
        
        if tid not in tile_needs:
            tile_needs[tid] = {"dem": False, "slope": False, "aspect": False, "geomorphon": False}
            
        if generate_dem and pd.isna(row.get("dem")) and not valid_raster(f"{base}.tif"):
            tile_needs[tid]["dem"] = True
        if generate_slope and pd.isna(row.get("slope")) and not valid_raster(slope_file):
            tile_needs[tid]["slope"] = True
        if generate_aspect and pd.isna(row.get("aspect")) and not valid_raster(aspect_file):
            tile_needs[tid]["aspect"] = True
        if generate_geomorphons and pd.isna(row.get("geomorphon")) and not valid_raster(geomorph_file):
            tile_needs[tid]["geomorphon"] = True
            
    tile_needs = {tid: needs for tid, needs in tile_needs.items() if any(needs.values())}
    logger.info(f"Found {len(tile_needs)} tiles that need processing")

    # Step 2 & 3: Only run if download_tiles is True
    downloaded = {}
    tile_results = {}
    if download_tiles:
        logger.info("Step 2: Downloading and preparing tiles...")
        for tid, needs in tqdm(tile_needs.items(), desc="Preparing tiles"):
            local_tif = os.path.join(dem_dir, f"{tid}.tif")
            if valid_raster(local_tif):
                downloaded[tid] = ([local_tif], "Local")
                continue
                
            m = re.match(r'([NS])(\d{2})([EW])(\d{3})', tid)
            if not m:
                logger.warning(f"Could not parse tile ID: {tid}")
                continue
                
            try:
                lat0 = int(m.group(2)) * (1 if m.group(1) == "N" else -1)
                lon0 = int(m.group(4)) * (1 if m.group(3) == "E" else -1)
                zip_paths, source = download_dem_point(lat0 + 0.5, lon0 + 0.5, out_dir=out_dir)
                if zip_paths:
                    tifs = [prepare_tif(zp) for zp in zip_paths]
                    # Move processed files to dem subfolder
                    moved_tifs = []
                    for tif in tifs:
                        target_path = os.path.join(dem_dir, f"{tid}.tif")
                        if tif != target_path:
                            import shutil
                            shutil.move(tif, target_path)
                        moved_tifs.append(target_path)
                    downloaded[tid] = (moved_tifs, source)
                else:
                    logger.warning(f"No DEM data available for tile {tid}")
                    stats["missing_tiles"] += 1
            except Exception as e:
                logger.error(f"Error downloading tile {tid}: {str(e)}")
                stats["missing_tiles"] += 1

        logger.info("Step 3: Running Whitebox processing...")
        for tid, (tifs, source) in tqdm(downloaded.items(), desc="Running Whitebox"):
            needs = tile_needs.get(tid, {})
            for tif in tifs:
                try:
                    tif_path, slope_path, aspect_path, geomorph_path = run_whitebox(
                        tif,
                        need_slope=generate_slope and needs.get("slope", False),
                        need_aspect=generate_aspect and needs.get("aspect", False),
                        need_geomorph=generate_geomorphons and needs.get("geomorphon", False),
                        slope_dir=slope_dir,
                        aspect_dir=aspect_dir,
                        geomorph_dir=geomorph_dir
                    )
                    tile_results[tid] = {
                        "tif": tif_path,
                        "slope": slope_path,
                        "aspect": aspect_path,
                        "geomorphon": geomorph_path,
                        "source": source
                    }
                    break
                except Exception as e:
                    logger.error(f"Whitebox processing failed for tile {tid}: {str(e)}")
                    continue

    # Step 4: Extract values from whatever exists
    logger.info("Step 4: Extracting values from rasters...")
    for i, row in tqdm(df.iterrows(), total=len(df), desc="Extracting values"):
        # Skip rows with invalid coordinates
        if i in invalid_coord_rows:
            continue
            
        lat, lon = row["y"], row["x"]
        tid = tile_id_from_coords(lat, lon)
        if tid is None:
            continue
            
        dem_base = os.path.join(dem_dir, tid)
        slope_base = os.path.join(slope_dir, f"{tid}_slope.tif")
        aspect_base = os.path.join(aspect_dir, f"{tid}_aspect.tif")
        geomorph_base = os.path.join(geomorph_dir, f"{tid}_geomorph.tif")
        
        tr = tile_results.get(tid, {
            "tif": f"{dem_base}.tif" if valid_raster(f"{dem_base}.tif") else None,
            "slope": slope_base if valid_raster(slope_base) else None,
            "aspect": aspect_base if valid_raster(aspect_base) else None,
            "geomorphon": geomorph_base if valid_raster(geomorph_base) else None,
            "source": "Local"
        })
        
        # Extract DEM value
        if generate_dem and pd.isna(row.get("dem")) and tr["tif"]:
            try:
                dem_value = extract_value(tr["tif"], lat, lon)
                if dem_value is not None:
                    df.at[i, "dem"] = dem_value
                    df.at[i, "dem_source"] = tr["source"]
                    stats["successful_dem"] += 1
                else:
                    df.at[i, "dem_failure"] = "No data value or out of bounds"
            except Exception as e:
                df.at[i, "dem_failure"] = f"Extraction error: {str(e)}"
                stats["extraction_failures"] += 1
        elif generate_dem and pd.isna(row.get("dem")):
            df.at[i, "dem_failure"] = "No DEM raster available"
        
        # Extract slope value
        if generate_slope and pd.isna(row.get("slope")) and tr["slope"]:
            try:
                slope_value = extract_value(tr["slope"], lat, lon)
                if slope_value is not None:
                    df.at[i, "slope"] = slope_value
                    stats["successful_slope"] += 1
                else:
                    df.at[i, "slope_failure"] = "No data value or out of bounds"
            except Exception as e:
                df.at[i, "slope_failure"] = f"Extraction error: {str(e)}"
                stats["extraction_failures"] += 1
        elif generate_slope and pd.isna(row.get("slope")):
            df.at[i, "slope_failure"] = "No slope raster available"
        
        # Extract aspect value
        if generate_aspect and pd.isna(row.get("aspect")) and tr["aspect"]:
            try:
                aspect_value = extract_value(tr["aspect"], lat, lon)
                if aspect_value is not None:
                    df.at[i, "aspect"] = aspect_value
                    stats["successful_aspect"] += 1
                else:
                    df.at[i, "aspect_failure"] = "No data value or out of bounds"
            except Exception as e:
                df.at[i, "aspect_failure"] = f"Extraction error: {str(e)}"
                stats["extraction_failures"] += 1
        elif generate_aspect and pd.isna(row.get("aspect")):
            df.at[i, "aspect_failure"] = "No aspect raster available"
        
        # Extract geomorphon value
        if generate_geomorphons and pd.isna(row.get("geomorphon")) and tr["geomorphon"]:
            try:
                geomorph_value = extract_value(tr["geomorphon"], lat, lon)
                if geomorph_value is not None:
                    df.at[i, "geomorphon"] = geomorph_value
                    stats["successful_geomorphon"] += 1
                else:
                    df.at[i, "geomorphon_failure"] = "No data value or out of bounds"
            except Exception as e:
                df.at[i, "geomorphon_failure"] = f"Extraction error: {str(e)}"
                stats["extraction_failures"] += 1
        elif generate_geomorphons and pd.isna(row.get("geomorphon")):
            df.at[i, "geomorphon_failure"] = "No geomorphon raster available"

    # Step 5: Decode geomorphons (only if geomorphons were generated)
    if generate_geomorphons:
        logger.info("Step 5: Decoding geomorphon classes...")
        geomorph_classes = {
            1: "flat", 2: "summit", 3: "ridge", 4: "shoulder", 5: "spur",
            6: "slope", 7: "hollow", 8: "footslope", 9: "valley", 10: "pit"
        }
        df["geomorphon_class"] = df["geomorphon"].map(geomorph_classes)

    # Save results
    df.to_csv(output_csv, index=False)
    
    # Print summary statistics
    logger.info("=" * 50)
    logger.info("PROCESSING SUMMARY")
    logger.info("=" * 50)
    logger.info(f"Total rows processed: {stats['total_rows']}")
    logger.info(f"Invalid coordinates: {stats['invalid_coords']}")
    logger.info(f"Missing tiles: {stats['missing_tiles']}")
    logger.info(f"Extraction failures: {stats['extraction_failures']}")
    
    # Show stats only for requested variables
    if generate_dem:
        logger.info(f"Successful DEM extractions: {stats['successful_dem']}")
    if generate_slope:
        logger.info(f"Successful slope extractions: {stats['successful_slope']}")
    if generate_aspect:
        logger.info(f"Successful aspect extractions: {stats['successful_aspect']}")
    if generate_geomorphons:
        logger.info(f"Successful geomorphon extractions: {stats['successful_geomorphon']}")
    
    logger.info(f"✅ Done! Saved {output_csv}")
    
    return stats

# ---------------------------
if __name__ == "__main__":
    enrich_csv(
        "data/negative_samples_within_land_10k_with_coords.csv",
        "data/negative_samples_within_land_10k_with_coords_topography.csv",
        out_dir="dem_tiles",  # Files will be organized in subfolders: dem/, slope/, aspect/, geomorphons/
        download_tiles=False,  # 🚨 Set to True if you want to download new tiles
        variables=["dem", "slope", "aspect", "geomorphons"]  # Specify which variables to append
    )



2025-09-15 19:25:02,937 - INFO - Loading CSV: data/negative_samples_within_land_10k_with_coords.csv
2025-09-15 19:25:02,948 - INFO - Loaded 5403 rows
2025-09-15 19:25:02,950 - INFO - Step 1: Scanning CSV for tile requirements...
Scanning CSV:  14%|█▍        | 754/5403 [00:05<00:30, 154.43it/s]2025-09-15 19:25:08,709 - INFO - GDAL signalled an error: err_no=1, msg='S28E016_aspect.tif: TIFFFetchDirectory:dem_tiles\\aspect\\S28E016_aspect.tif: Can not read TIFF directory count'
2025-09-15 19:25:08,710 - INFO - GDAL signalled an error: err_no=1, msg='S28E016_aspect.tif: TIFFReadDirectory:Failed to read directory at offset 51840008'
Scanning CSV:  15%|█▍        | 803/5403 [00:06<00:32, 142.96it/s]2025-09-15 19:25:09,012 - INFO - GDAL signalled an error: err_no=1, msg='N53E039_aspect.tif: TIFFFetchDirectory:dem_tiles\\aspect\\N53E039_aspect.tif: Can not read TIFF directory count'
2025-09-15 19:25:09,014 - INFO - GDAL signalled an error: err_no=1, msg='N53E039_aspect.tif: TIFFReadDirectory:Fa

# Append to vector file

In [ ]:
import os
import re
import zipfile
import numpy as np
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.transform import from_origin
from whitebox.whitebox_tools import WhiteboxTools
import earthaccess
from tqdm import tqdm
import contextlib
import io
import math
import shutil

# ---------------------------
# Tile ID utilities
# ---------------------------
def tile_id_from_coords(lat, lon):
    """Convert coords to tile ID (e.g. N40W106)."""
    if pd.isna(lat) or pd.isna(lon):
        return None
    ns = "N" if lat >= 0 else "S"
    ew = "E" if lon >= 0 else "W"
    lat_tile = math.floor(lat)
    lon_tile = math.floor(lon)
    return f"{ns}{abs(lat_tile):02d}{ew}{abs(lon_tile):03d}"

# ---------------------------
# DEM Download
# ---------------------------
def download_dem_bbox(min_lon, min_lat, max_lon, max_lat, out_dir="dem_tiles", prefer="SRTMGL1"):
    os.makedirs(out_dir, exist_ok=True)
    earthaccess.login(strategy="environment", persist=True)
    dataset = ("SRTMGL1", "003") if prefer == "SRTMGL1" else ("COPDEM_GLO_30", "001")
    try:
        results = earthaccess.search_data(
            short_name=dataset[0],
            version=dataset[1],
            bounding_box=(min_lon, min_lat, max_lon, max_lat),
            count=10
        )
    except IndexError:
        return []
    if not results or len(results) == 0:
        return []
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf), contextlib.redirect_stderr(buf):
        paths = earthaccess.download(results, out_dir)
    return paths

def download_dem_point(lat, lon, out_dir="dem_tiles", buffer=0.1):
    # Clamp bbox
    min_lon = max(-180.0, lon - buffer)
    max_lon = min(180.0, lon + buffer)
    min_lat = max(-90.0, lat - buffer)
    max_lat = min(90.0, lat + buffer)

    # Try SRTM first (only valid between -56 and +60 lat)
    if -56 <= lat <= 60:
        paths = download_dem_bbox(min_lon, min_lat, max_lon, max_lat, out_dir=out_dir, prefer="SRTMGL1")
        if paths:
            return paths, "SRTM"

    # Fallback: Copernicus global DEM
    paths = download_dem_bbox(min_lon, min_lat, max_lon, max_lat, out_dir=out_dir, prefer="COPDEM")
    if paths:
        return paths, "Copernicus"
    return [], "None"

# ---------------------------
# HGT → GeoTIFF
# ---------------------------
def parse_hgt_bounds(hgt_path):
    name = os.path.splitext(os.path.basename(hgt_path))[0]
    m = re.match(r'([NS])(\d{1,2})([EW])(\d{1,3})', name, re.IGNORECASE)
    if not m:
        raise ValueError(f"Cannot parse HGT name: {hgt_path}")
    lat_sign = 1 if m.group(1).upper() == 'N' else -1
    lon_sign = 1 if m.group(3).upper() == 'E' else -1
    lat0 = lat_sign * int(m.group(2))
    lon0 = lon_sign * int(m.group(4))
    west, south = float(lon0), float(lat0)
    east, north = west + 1.0, south + 1.0
    return west, south, east, north

def hgt_to_gtiff(hgt_path, tif_path):
    west, south, east, north = parse_hgt_bounds(hgt_path)
    nbytes = os.path.getsize(hgt_path)
    side = int(np.sqrt(nbytes // 2))
    if side not in (3601, 1201):
        raise ValueError(f"Unexpected HGT side length: {side}")
    data = np.fromfile(hgt_path, dtype=">i2").reshape((side, side))
    data = data[:-1, :-1]
    res = 1.0 / (side - 1)
    transform = from_origin(west, north, res, res)
    profile = {
        "driver": "GTiff",
        "height": data.shape[0],
        "width": data.shape[1],
        "count": 1,
        "dtype": "int16",
        "crs": "EPSG:4326",
        "transform": transform,
        "nodata": -32768,
        "tiled": True,
        "compress": "LZW"
    }
    with rasterio.open(tif_path, "w", **profile) as dst:
        dst.write(data, 1)

def prepare_tif(path):
    """Unpack zip/HGT and convert to GeoTIFF. Remove raw files after processing."""
    if path.lower().endswith(".tif"):
        return os.path.abspath(path)
    if path.lower().endswith(".zip"):
        tif_out, hgt_out = None, None
        with zipfile.ZipFile(path, "r") as z:
            tifs = [m for m in z.namelist() if m.lower().endswith(".tif")]
            if tifs:
                tif_out = os.path.join(os.path.dirname(path), os.path.basename(tifs[0]))
                if not os.path.exists(tif_out):
                    z.extract(tifs[0], os.path.dirname(path))
                tif_out = os.path.abspath(tif_out)
            else:
                hgts = [m for m in z.namelist() if m.lower().endswith(".hgt")]
                if hgts:
                    hgt_out = os.path.join(os.path.dirname(path), os.path.basename(hgts[0]))
                    if not os.path.exists(hgt_out):
                        z.extract(hgts[0], os.path.dirname(path))
                    tif_out = hgt_out.replace(".hgt", ".tif")
                    if not os.path.exists(tif_out):
                        hgt_to_gtiff(hgt_out, tif_out)
                    try:
                        os.remove(hgt_out)
                    except PermissionError:
                        pass
                    tif_out = os.path.abspath(tif_out)
        try:
            os.remove(path)
        except PermissionError:
            pass
        if tif_out:
            return tif_out
        else:
            raise FileNotFoundError(f"No .tif or .hgt in {path}")
    raise FileNotFoundError(f"Unsupported DEM format: {path}")

# ---------------------------
# Whitebox + helpers
# ---------------------------
wbt = WhiteboxTools()
wbt.verbose = False

def valid_raster(path):
    if not path or not os.path.exists(path) or os.path.getsize(path) == 0:
        return False
    try:
        with rasterio.open(path) as src:
            _ = src.count
        return True
    except Exception:
        return False

def run_whitebox(tif_file, slope_dir, aspect_dir, geomorph_dir,
                 need_slope=True, need_aspect=True, need_geomorph=True):
    tif_file = os.path.abspath(tif_file).replace("\\", "/")
    base_name = os.path.splitext(os.path.basename(tif_file))[0]

    slope_tif = os.path.join(slope_dir, f"{base_name}_slope.tif")
    aspect_tif = os.path.join(aspect_dir, f"{base_name}_aspect.tif")
    geomorph_tif = os.path.join(geomorph_dir, f"{base_name}_geomorph.tif")

    if need_slope and not valid_raster(slope_tif):
        wbt.slope(dem=tif_file, output=slope_tif, zfactor=1.0, units="degrees")
    if need_aspect and not valid_raster(aspect_tif):
        wbt.aspect(dem=tif_file, output=aspect_tif)
    if need_geomorph and not valid_raster(geomorph_tif):
        wbt.geomorphons(dem=tif_file, output=geomorph_tif, search=50, threshold=0.0, forms=True)

    return tif_file, slope_tif, aspect_tif, geomorph_tif

# ---------------------------
# Extract raster value
# ---------------------------
def extract_value(raster, lat, lon):
    if not valid_raster(raster):
        return None
    try:
        with rasterio.open(raster) as src:
            nd = src.nodata
            for val in src.sample([(lon, lat)]):
                v = float(val[0])
                if np.isnan(v) or (nd is not None and v == nd):
                    return None
                return v
    except Exception:
        return None

# ---------------------------
# Main pipeline for GeoJSON
# ---------------------------
def enrich_geojson(input_geojson, output_geojson, out_dir="dem_tiles", download_tiles=True):
    os.makedirs(out_dir, exist_ok=True)

    # Create subfolders
    dem_dir = os.path.join(out_dir, "dem")
    slope_dir = os.path.join(out_dir, "slope")
    aspect_dir = os.path.join(out_dir, "aspect")
    geomorph_dir = os.path.join(out_dir, "geomorphons")
    for d in [dem_dir, slope_dir, aspect_dir, geomorph_dir]:
        os.makedirs(d, exist_ok=True)

    gdf = gpd.read_file(input_geojson)

    # Ensure WGS84
    if gdf.crs is None:
        print("⚠️ No CRS found, assuming EPSG:4326")
        gdf.set_crs(epsg=4326, inplace=True)
    else:
        gdf = gdf.to_crs(epsg=4326)

    # Filter SRTM coverage (lat -56 to 60)
    gdf = gdf[gdf.geometry.centroid.y.between(-56, 60)]

    # Add expected cols
    for col in ["dem", "slope", "aspect", "geomorphon", "dem_source", "geomorphon_class"]:
        if col not in gdf.columns:
            gdf[col] = None

    # Collect centroids
    centroids = gdf.geometry.centroid
    coords = [(pt.y, pt.x) for pt in centroids]

    # Step 1: collect tiles
    needed_tiles = {}
    for (lat, lon) in tqdm(coords, desc="Collecting tiles"):
        tid = tile_id_from_coords(lat, lon)
        if tid and tid not in needed_tiles:
            needed_tiles[tid] = (lat, lon)

    # Step 2: download & prepare tiles
    downloaded = {}
    for tid, (lat, lon) in tqdm(needed_tiles.items(), desc="Preparing tiles"):
        tif_path = os.path.join(dem_dir, f"{tid}.tif")
        if valid_raster(tif_path):
            downloaded[tid] = ([tif_path], "Local")
        elif download_tiles:
            zip_paths, source = download_dem_point(lat, lon, out_dir=out_dir)
            if zip_paths:
                tifs = [prepare_tif(zp) for zp in zip_paths]
                moved_tifs = []
                for tif in tifs:
                    target = os.path.join(dem_dir, f"{tid}.tif")
                    if tif != target:
                        shutil.move(tif, target)
                    moved_tifs.append(target)
                downloaded[tid] = (moved_tifs, source)

    # Step 3: run Whitebox
    tile_results = {}
    for tid, (tifs, source) in tqdm(downloaded.items(), desc="Running Whitebox"):
        for tif in tifs:
            tif_path, slope_tif, aspect_tif, geomorph_tif = run_whitebox(
                tif, slope_dir, aspect_dir, geomorph_dir)
            tile_results[tid] = (tif_path, slope_tif, aspect_tif, geomorph_tif, source)

    # Step 4: extract values
    geomorph_classes = {
        1: "flat", 2: "summit", 3: "ridge", 4: "shoulder", 5: "spur",
        6: "slope", 7: "hollow", 8: "footslope", 9: "valley", 10: "pit"
    }

    for idx, (lat, lon) in enumerate(tqdm(coords, desc="Extracting values")):
        tid = tile_id_from_coords(lat, lon)
        if tid is None or tid not in tile_results:
            continue
        tif, slope_tif, aspect_tif, geomorph_tif, source = tile_results[tid]
        gdf.at[idx, "dem"] = extract_value(tif, lat, lon)
        gdf.at[idx, "dem_source"] = source
        gdf.at[idx, "slope"] = extract_value(slope_tif, lat, lon)
        gdf.at[idx, "aspect"] = extract_value(aspect_tif, lat, lon)
        gdf.at[idx, "geomorphon"] = extract_value(geomorph_tif, lat, lon)
        gdf.at[idx, "geomorphon_class"] = geomorph_classes.get(gdf.at[idx, "geomorphon"], None)

    # Save enriched GeoJSON
    gdf.to_file(output_geojson, driver="GeoJSON")
    print(f"✅ Done! Saved {output_geojson}")

# ---------------------------
# Run
# ---------------------------
if __name__ == "__main__":
    enrich_geojson(
        "data/grid_tuscany_forest.geojson",
        "data/grid_tuscany_with_topography.geojson",
        out_dir="dem_tiles",
        download_tiles=True
    )


# Upload to gcp

In [8]:
# Upload DEM tiles to Google Cloud Storage
import os
from google.cloud import storage
from tqdm import tqdm
import glob

def upload_dem_tiles_to_gcs(local_dir="dem_tiles", bucket_name="your-bucket-name", gcs_prefix="dem_tiles/"):
    """
    Upload all DEM tiles from local directory to Google Cloud Storage bucket.
    
    Args:
        local_dir: Local directory containing DEM tiles
        bucket_name: Name of the GCS bucket
        gcs_prefix: Prefix for files in the bucket (e.g., "dem_tiles/")
    
    Returns:
        List of uploaded file names
    """
    # Initialize the GCS client
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    
    # Find all .tif files in the local directory
    pattern = os.path.join(local_dir, "**", "*.tif")
    local_files = glob.glob(pattern, recursive=True)
    
    if not local_files:
        print(f"No .tif files found in {local_dir}")
        return []
    
    print(f"Found {len(local_files)} .tif files to upload")
    
    uploaded_files = []
    failed_uploads = []
    
    for local_file in tqdm(local_files, desc="Uploading to GCS"):
        try:
            # Create the destination blob name
            relative_path = os.path.relpath(local_file, local_dir)
            blob_name = gcs_prefix + relative_path.replace("\\", "/")  # Ensure forward slashes
            
            # Create blob and upload
            blob = bucket.blob(blob_name)
            blob.upload_from_filename(local_file)
            
            uploaded_files.append(blob_name)
            print(f"✅ Uploaded: {blob_name}")
            
        except Exception as e:
            failed_uploads.append((local_file, str(e)))
            print(f"❌ Failed to upload {local_file}: {str(e)}")
    
    # Summary
    print(f"\n📊 Upload Summary:")
    print(f"✅ Successfully uploaded: {len(uploaded_files)} files")
    print(f"❌ Failed uploads: {len(failed_uploads)} files")
    
    if failed_uploads:
        print("\nFailed uploads:")
        for file_path, error in failed_uploads:
            print(f"  - {file_path}: {error}")
    
    return uploaded_files

def download_dem_tiles_from_gcs(bucket_name="your-bucket-name", gcs_prefix="dem_tiles/", local_dir="dem_tiles"):
    """
    Download DEM tiles from Google Cloud Storage bucket to local directory.
    
    Args:
        bucket_name: Name of the GCS bucket
        gcs_prefix: Prefix for files in the bucket (e.g., "dem_tiles/")
        local_dir: Local directory to download files to
    
    Returns:
        List of downloaded file names
    """
    # Initialize the GCS client
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    
    # List all blobs with the given prefix
    blobs = bucket.list_blobs(prefix=gcs_prefix)
    
    # Filter for .tif files
    tif_blobs = [blob for blob in blobs if blob.name.endswith('.tif')]
    
    if not tif_blobs:
        print(f"No .tif files found in bucket {bucket_name} with prefix {gcs_prefix}")
        return []
    
    print(f"Found {len(tif_blobs)} .tif files to download")
    
    downloaded_files = []
    failed_downloads = []
    
    for blob in tqdm(tif_blobs, desc="Downloading from GCS"):
        try:
            # Create local file path
            relative_path = blob.name[len(gcs_prefix):]  # Remove prefix
            local_file = os.path.join(local_dir, relative_path)
            
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(local_file), exist_ok=True)
            
            # Download the file
            blob.download_to_filename(local_file)
            
            downloaded_files.append(local_file)
            print(f"✅ Downloaded: {blob.name}")
            
        except Exception as e:
            failed_downloads.append((blob.name, str(e)))
            print(f"❌ Failed to download {blob.name}: {str(e)}")
    
    # Summary
    print(f"\n📊 Download Summary:")
    print(f"✅ Successfully downloaded: {len(downloaded_files)} files")
    print(f"❌ Failed downloads: {len(failed_downloads)} files")
    
    if failed_downloads:
        print("\nFailed downloads:")
        for blob_name, error in failed_downloads:
            print(f"  - {blob_name}: {error}")
    
    return downloaded_files

def list_gcs_dem_tiles(bucket_name="your-bucket-name", gcs_prefix="dem_tiles/"):
    """
    List all DEM tiles in the GCS bucket.
    
    Args:
        bucket_name: Name of the GCS bucket
        gcs_prefix: Prefix for files in the bucket (e.g., "dem_tiles/")
    
    Returns:
        List of blob names
    """
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    
    blobs = bucket.list_blobs(prefix=gcs_prefix)
    tif_blobs = [blob.name for blob in blobs if blob.name.endswith('.tif')]
    
    print(f"Found {len(tif_blobs)} .tif files in bucket {bucket_name}")
    for blob_name in tif_blobs:
        print(f"  - {blob_name}")
    
    return tif_blobs

# Example usage:
if __name__ == "__main__":
    # Set your bucket name
    BUCKET_NAME = "mushroom-radar"
    
    # Upload DEM tiles to GCS
    print("Uploading DEM tiles to Google Cloud Storage...")
    uploaded = upload_dem_tiles_to_gcs(
        local_dir="dem_tiles",
        bucket_name=BUCKET_NAME,
        gcs_prefix="dem_tiles/"
    )
    
    # List files in bucket
    print("\nListing files in bucket...")
    list_gcs_dem_tiles(bucket_name=BUCKET_NAME, gcs_prefix="dem_tiles/")
    
    # Download DEM tiles from GCS (example)
    # print("\nDownloading DEM tiles from Google Cloud Storage...")
    # downloaded = download_dem_tiles_from_gcs(
    #     bucket_name=BUCKET_NAME,
    #     gcs_prefix="dem_tiles/",
    #     local_dir="dem_tiles_downloaded"
    # )


Uploading DEM tiles to Google Cloud Storage...
Found 15375 .tif files to upload


Uploading to GCS:   0%|          | 1/15375 [00:00<2:44:39,  1.56it/s]

✅ Uploaded: dem_tiles/N00E009.tif


Uploading to GCS:   0%|          | 2/15375 [00:01<2:23:46,  1.78it/s]

✅ Uploaded: dem_tiles/N00E010.tif


...

✅ Uploaded: dem_tiles/N00W059.tif


Uploading to GCS:   1%|          | 91/15375 [01:34<5:16:20,  1.24s/it]

✅ Uploaded: dem_tiles/N00W059_aspect.tif


Uploading to GCS:   1%|          | 91/15375 [01:35<4:26:09,  1.04s/it]


KeyboardInterrupt: 